[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AccelerationConsortium/wireless-color-sensor/blob/main/sensor_file/test_sensor.ipynb)

In [ ]:
%pip install paho-mqtt

In [ ]:
import json
import ssl
import time
from uuid import uuid4
import paho.mqtt.client as mqtt

# MQTT Broker Configuration
HIVEMQ_HOST = "248cc294c37642359297f75b7b023374.s2.eu.hivemq.cloud"  # @param {type:"string"}
HIVEMQ_PORT = 8883  # @param {type:"integer"}
HIVEMQ_USERNAME = "sgbaird"  # @param {type:"string"}
HIVEMQ_PASSWORD = "D.Pq5gYtejYbU#L"  # @param {type:"string"}

# Sensor Configuration
PICO_ID = "test"  # @param {type:"string"}

# MQTT Topics (matches sensor_file/main.py)
COMMAND_TOPIC = f"command/picow/{PICO_ID}/as7341/read"
SENSOR_DATA_TOPIC = f"color-mixing/picow/{PICO_ID}/as7341"

sensor_data_received = None


def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe(SENSOR_DATA_TOPIC, qos=1)


def on_message(client, userdata, msg):
    global sensor_data_received
    payload = json.loads(msg.payload.decode())
    print(f"Received: {json.dumps(payload, indent=2)}")
    sensor_data_received = payload


client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
client.tls_set(tls_version=ssl.PROTOCOL_TLS_CLIENT)
client.username_pw_set(HIVEMQ_USERNAME, HIVEMQ_PASSWORD)
client.on_connect = on_connect
client.on_message = on_message
client.connect(HIVEMQ_HOST, HIVEMQ_PORT)
client.loop_start()
time.sleep(2)

In [ ]:
# Send a test command to the color sensor
R = 50  # @param {type:"integer"}
Y = 50  # @param {type:"integer"}
B = 50  # @param {type:"integer"}

experiment_id = str(uuid4())
payload = {
    "command": {"R": R, "Y": Y, "B": B},
    "experiment_id": experiment_id,
}

print(f"Sending: {json.dumps(payload)}")
client.publish(COMMAND_TOPIC, json.dumps(payload), qos=1)

# Wait for response
for _ in range(30):
    time.sleep(1)
    if sensor_data_received and sensor_data_received.get("experiment_id") == experiment_id:
        print("\nSensor data received successfully!")
        break
else:
    print("\nTimeout waiting for sensor data")

In [ ]:
client.loop_stop()
client.disconnect()
print("Disconnected")